In [ ]:
# I need to find which speakers are within a subtitle time range. 
# I have a list of speakers and timestamps in /work/inga/data/h2/ruv-di/<filename>/ruvdi_segments, let's call them diarization segments, and subtitle timestamps in /work/inga/data/h2/asr_ruvdi/transcripts/<filename>/segments
# I have to find within which diarization timestamps the subtitle timestamps lie and assign the corresponding spkID to the subtitle timestamp 
# NOTE! Before i start assigning spkIDs. I need to use reco2spk_num2spk_info.csv to fix the speaker IDs in the ruv-di data, and then put all the segment data into one file

In [7]:
import pandas as pd

In [8]:

meta = pd.read_table('/home/staff/inga/h2/data/reco2spk_num2spk_info.csv', header=None, sep=',')
diar = pd.read_table('/home/staff/inga/h2/data/ruv-di/all_segments', header=None, sep='\s+')

In [26]:
print(meta.head(n=10))
print(diar.head(n=15))

0   1                            2        3
0  4886083R7   1        Bergsteinn Sigurðsson  SPK0016
1  4886083R7   2  Steinþór Helgi Arnsteinsson  SPK0129
2  4886083R7   3                     Female 1  UNK0003
3  4886083R7   4      Menningin Interviewer 1  UNK0020
4  4886083R7   5           Guðbergur Bergsson  SPK0050
5  4886083R7   6       Helga Braga Jónsdóttir  SPK0065
6  4886083R7   7       Kastljós Interviewer 1  UNK0014
7  4886083R7   8         Ævar Rafn Hafþórsson  SPK0002
8  4886083R7   9          Ilmur Stefánsdóttir  SPK0067
9  4886083R7  10              Kastljós Host 1  UNK0011
     0                1   2          3       4       5
0   10  4886083R7_00000  10  4886083R7   17.27   19.52
1   10  4886083R7_00001  10  4886083R7   19.52   64.55
2   10  4886083R7_00002  10  4886083R7   64.55   65.24
3   10  4886083R7_00003  10  4886083R7   65.24   81.17
4   10  4886083R7_00004  10  4886083R7   81.92   97.45
5    3  4886083R7_00005   3  4886083R7   98.20  105.78
6    6  4886083R7_000

In [23]:
def fix_spkID(meta,diar,seg):
    '''Map the infile speaker IDs to the overall ones'''
    for diar_row in diar.itertuples(index=False):
        for meta_row in meta.itertuples(index=False):
            if meta_row[0] == diar_row[3] and meta_row[1] == diar_row[0]:
                seg.write(f"{meta_row[3]}-{diar_row[1]} {meta_row[3]}-{diar_row[3]} {diar_row[4]} {diar_row[5]}\n")
                break

In [24]:
with open('/home/staff/inga/h2/data/ruv-di/all_segments_wspkID','w') as seg:
    fix_spkID(meta,diar,seg)

In [ ]:
# Now I can pair my speaker identified diarization segments with the new segments obtained from audio-subtitle text alignment, which need speaker IDs. 

In [9]:
sub = pd.read_table('/work/inga/data/h2/judy_reseg/segments', header=None, sep='\s+')
spkdi = pd.read_table('/home/staff/inga/h2/data/ruv-di/all_segments_wspkID', header=None, sep='\s+')

In [10]:
print(sub.head(n=5))
print(spkdi.head(n=5))

                                         0                  1      2      3
0  unknown-4886083R7-00000000-00003000-1-1  unknown-4886083R7  17.74  25.28
1  unknown-4886083R7-00000000-00003000-2-1  unknown-4886083R7  24.85  29.99
2  unknown-4886083R7-00002500-00005500-1-1  unknown-4886083R7  30.02  32.22
3  unknown-4886083R7-00002500-00005500-1-2  unknown-4886083R7  32.51  33.53
4  unknown-4886083R7-00002500-00005500-2-1  unknown-4886083R7  33.44  41.91
                         0                  1      2      3
0  UNK0011-4886083R7_00000  UNK0011-4886083R7  17.27  19.52
1  UNK0011-4886083R7_00001  UNK0011-4886083R7  19.52  64.55
2  UNK0011-4886083R7_00002  UNK0011-4886083R7  64.55  65.24
3  UNK0011-4886083R7_00003  UNK0011-4886083R7  65.24  81.17
4  UNK0011-4886083R7_00004  UNK0011-4886083R7  81.92  97.45


In [11]:
ids = sub[1].unique()
recoid = [id.split("-")[1] for id in ids]
print(recoid)

['4886083R7', '4886107R8', '4886131R9', '4898511R7', '4898535R3', '4930605S0', '4934417T0', '4934441T0', '4934466T0', '4959359T0', '4959383T0', '4959408T0', '5004302T0', '5012565T0', '5019095T0', '5021994T0', '5021995T0', '5022004T0', '5022010T0', '5022028T0']


In [19]:
def extrack_spk(sub,spkdi,seg, strict: bool):
    ids = sub[1].unique()
    recoid_list = [id.split("-")[1] for id in ids] # List of all recording IDs
    index_list = []
    for recoid in recoid_list:
        # Create new dataframes with only lines containing this recording ID
        spkdi_part = spkdi[spkdi[1].str.contains(recoid)]
        sub_part = sub[sub[1].str.contains(recoid)]
        #start = 0
        for sub_row in sub_part.itertuples(index=False):
            # Find all rows is diarization data that have segment start before my subtitle start and segment end after it
            if strict:
                dirows = spkdi_part.loc[(spkdi_part[2] <= sub_row[2]) & ((spkdi_part[3] + 1) >= sub_row[3])]
            else:
                dirows = spkdi_part.loc[(spkdi_part[2] <= sub_row[2]) & (spkdi_part[3] >= sub_row[2])]
            if not dirows.empty:
                spkid = str(dirows[1]).split("-")[0] # Extract the spkID from the uttID
                spkid2 = spkid.split()[1]
                sub_recoid, count = sub_row[0].split("-")[1:3]
                seg.write(f"{spkid2}-{sub_recoid}-{count} {spkid2}-{sub_recoid} {sub_row[2]} {sub_row[3]}\n")
                #index_list.append([start, spkid, recoid, sub_recoid, count, sub_row[2:4], dirows])
            #start += 1
    #return index_list


In [20]:
strict = True
with open('/work/inga/data/h2/judy_reseg/segments_wspkid_strict2','w') as seg:
    extrack_spk(sub,spkdi,seg,strict=True)

In [83]:
print(len(outlist))
print(len(sub))

1514
2304


In [76]:
outlist[356:401]

[[49,
  '228    UNK0018',
  '4886131R9',
  '4886131R9',
  '050',
  (814.14, 823.211),
                             0                  1      2       3
  228  UNK0018-4886131R9_00040  UNK0018-4886131R9  803.6  814.85],
 [50,
  '231    SPK0094',
  '4886131R9',
  '4886131R9',
  '051',
  (823.57, 824.471),
                             0                  1       2       3
  231  SPK0094-4886131R9_00043  SPK0094-4886131R9  819.48  829.98],
 [51,
  '231    SPK0094',
  '4886131R9',
  '4886131R9',
  '052',
  (825.24, 831.3610000000001),
                             0                  1       2       3
  231  SPK0094-4886131R9_00043  SPK0094-4886131R9  819.48  829.98],
 [52,
  '232    UNK0018',
  '4886131R9',
  '4886131R9',
  '053',
  (836.46, 837.3910000000001),
                             0                  1       2       3
  232  UNK0018-4886131R9_00044  UNK0018-4886131R9  829.98  898.25],
 [53,
  '232    UNK0018',
  '4886131R9',
  '4886131R9',
  '054',
  (837.58, 839.291),
                